# Credit Score Classification

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [118]:
#import test and training data
traindata=pd.read_csv(r'E:\LearningPython\Neural Networks\Credit Score\Dataset\archive\train.csv', low_memory=False)
testdata=pd.read_csv(r'E:\LearningPython\Neural Networks\Credit Score\Dataset\archive\test.csv', low_memory=False)

In [119]:
pd.set_option('display.max_columns', 35)

In [120]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_recall_curve, ConfusionMatrixDisplay, accuracy_score
from imblearn.over_sampling import SMOTE

In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# Data Study

In [122]:
traindata.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [123]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

##### Change type to category

In [124]:
catcols=['Month', 'Occupation', 'Type_of_Loan', 'Credit_Mix', 'Credit_Score', 'Payment_Behaviour', 'Payment_of_Min_Amount']
traindata[catcols]=traindata[catcols].astype('category')

# Data Cleaning

In [125]:
traindata.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [126]:
traindata.duplicated().sum()

0

In [127]:
traindata.nunique()

ID                          100000
Customer_ID                  12500
Month                            8
Name                         10139
Age                           1788
SSN                          12501
Occupation                      16
Annual_Income                18940
Monthly_Inhand_Salary        13235
Num_Bank_Accounts              943
Num_Credit_Card               1179
Interest_Rate                 1750
Num_of_Loan                    434
Type_of_Loan                  6260
Delay_from_due_date             73
Num_of_Delayed_Payment         749
Changed_Credit_Limit          4384
Num_Credit_Inquiries          1223
Credit_Mix                       4
Outstanding_Debt             13178
Credit_Utilization_Ratio    100000
Credit_History_Age             404
Payment_of_Min_Amount            3
Total_EMI_per_month          14950
Amount_invested_monthly      91049
Payment_Behaviour                7
Monthly_Balance              98792
Credit_Score                     3
dtype: int64

#### Drop redundant columns


In [128]:
traindata=traindata.drop(['ID', 'Name', 'SSN', 'Type_of_Loan'], axis=1)


##### Handle categorical columns

In [129]:
cl=list(traindata.select_dtypes('category').columns)

In [130]:
for col in cl:
    print(col)
    print ((traindata[col].value_counts()))
    print()

Month
April       12500
August      12500
February    12500
January     12500
July        12500
June        12500
March       12500
May         12500
Name: Month, dtype: int64

Occupation
_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: Occupation, dtype: int64

Credit_Mix
Standard    36479
Good        24337
_           20195
Bad         18989
Name: Credit_Mix, dtype: int64

Payment_of_Min_Amount
Yes    52326
No     35667
NM     12007
Name: Payment_of_Min_Amount, dtype: int64

Payment_Behaviour
Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_

For credit mix replace _ with NaN 

For payment_behavior replace !@9#%8 with NaN

In [131]:
traindata['Payment_Behaviour']=traindata['Payment_Behaviour'].apply(lambda x: np.NaN if x=='!@9#%8' else x )
traindata['Credit_Mix']=traindata['Credit_Mix'].apply(lambda x: np.NaN if x=='_' else x )

In [132]:
traindata['Occupation']=traindata.groupby('Customer_ID')['Occupation'].transform(lambda x: x.mode()[0])

In [133]:
traindata=traindata[~(traindata['Occupation']=='_______')]

##### Handle Numeric Variables

In [134]:
cols=['Age','Annual_Income','Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit', 'Outstanding_Debt']

In [135]:
for col in cols:
    traindata[col]=traindata[col].apply(lambda x:x if x is np.NaN or not isinstance(x,str) else x.replace('_','')).replace('',np.NaN)
    traindata[col]=traindata[col].astype('float')

##### Impute and Clean

In [136]:
traindata['Monthly_Inhand_Salary']=traindata.groupby('Customer_ID')['Monthly_Inhand_Salary'].transform(lambda v: v.mean())


In [137]:
traindata['Num_of_Delayed_Payment'] = traindata.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda v: v.mode()[0])


In [138]:
def return_null(val):
  if isinstance(val, str) and "__" in val:
    return np.NaN
  else:
    return val
traindata['Amount_invested_monthly'] = traindata['Amount_invested_monthly'].apply(return_null)
traindata['Amount_invested_monthly'] = traindata['Amount_invested_monthly'].astype('float')
traindata['Amount_invested_monthly'] = traindata.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda v: v.mean())

In [139]:
traindata['Num_Credit_Inquiries']=traindata.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.mean())

In [140]:
traindata['Monthly_Balance'] = traindata['Monthly_Balance'].apply(return_null)
traindata['Monthly_Balance'] = traindata['Monthly_Balance'].astype('float')
traindata['Monthly_Balance'] =traindata.groupby('Customer_ID')['Monthly_Balance'].transform(lambda v: v.mean())

In [141]:
traindata['Payment_Behaviour'] = traindata.groupby('Customer_ID')['Payment_Behaviour'].apply(lambda x: x.fillna(x.mode()[0]))
traindata['Credit_Mix'] = traindata.groupby('Customer_ID')['Credit_Mix'].apply(lambda x: x.fillna(x.mode()[0]))
traindata['Changed_Credit_Limit'] = traindata.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda v: v.mode()[0])

C:\Users\user\AppData\Local\Temp\ipykernel_16784\340677367.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  traindata['Payment_Behaviour'] = traindata.groupby('Customer_ID')['Payment_Behaviour'].apply(lambda x: x.fillna(x.mode()[0]))
C:\Users\user\AppData\Local\Temp\ipykernel_16784\340677367.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this w

In [142]:
traindata['Payment_Behaviour'].isna().sum()

0

In [143]:
import re
def cal_history_age(val):
  try:
    year, month = 0, 0
    if re.search('year', val, re.IGNORECASE):
      year = int(re.findall('\d+', val)[0])
    if re.search('month', val, re.IGNORECASE):
      month = int(re.findall('\d+', val)[1])
    return year*12+month
  except:
    return np.NaN

In [144]:
traindata['Credit_History_Age']=traindata['Credit_History_Age'].apply(cal_history_age)

In [145]:
traindata['Credit_History_Age']=traindata.groupby('Customer_ID')['Credit_History_Age'].transform(lambda v: v.ffill())
traindata['Credit_History_Age']=traindata.groupby('Customer_ID')['Credit_History_Age'].transform(lambda v: v.bfill())
traindata['Credit_History_Age']=traindata['Credit_History_Age'].astype('float')

In [146]:
traindata.select_dtypes('category').columns

Index(['Month', 'Payment_of_Min_Amount', 'Credit_Score'], dtype='object')

In [147]:
traindata.select_dtypes('object')

,Customer_ID,Occupation,Credit_Mix,Payment_Behaviour
0,CUS_0xd40,Scientist,Good,High_spent_Small_value_payments
1,CUS_0xd40,Scientist,Good,Low_spent_Large_value_payments
2,CUS_0xd40,Scientist,Good,Low_spent_Medium_value_payments
3,CUS_0xd40,Scientist,Good,Low_spent_Small_value_payments
4,CUS_0xd40,Scientist,Good,High_spent_Medium_value_payments
...,...,...,...,...
99995,CUS_0x942c,Mechanic,Good,High_spent_Large_value_payments
99996,CUS_0x942c,Mechanic,Good,High_spent_Medium_value_payments
99997,CUS_0x942c,Mechanic,Good,High_spent_Large_value_payments
99998,CUS_0x942c,Mechanic,Good,Low_spent_Large_value_payments


In [148]:
cat_cols = ['Credit_Mix', 'Payment_Behaviour', 'Occupation']
traindata[cat_cols] = traindata[cat_cols].astype('category')


In [149]:
def clean_age(x):
  if x > 100 or x <=0:
    return np.NaN
  return x

In [150]:
traindata['Age'] = traindata['Age'].apply(lambda x: x if x is np.nan or not isinstance(x, str)
                    else x.replace("_","")).replace("",np.nan)
traindata['Age'] = traindata['Age'].astype('float')
traindata['Age'] = traindata['Age'].apply(clean_age)
traindata['Age'] = traindata.groupby('Customer_ID')['Age'].transform(lambda v: v.mode()[0])

##### Sanity Checks

In [151]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Customer_ID               99992 non-null  object  
 1   Month                     99992 non-null  category
 2   Age                       99992 non-null  float64 
 3   Occupation                99992 non-null  category
 4   Annual_Income             99992 non-null  float64 
 5   Monthly_Inhand_Salary     99992 non-null  float64 
 6   Num_Bank_Accounts         99992 non-null  int64   
 7   Num_Credit_Card           99992 non-null  int64   
 8   Interest_Rate             99992 non-null  int64   
 9   Num_of_Loan               99992 non-null  float64 
 10  Delay_from_due_date       99992 non-null  int64   
 11  Num_of_Delayed_Payment    99992 non-null  float64 
 12  Changed_Credit_Limit      99992 non-null  float64 
 13  Num_Credit_Inquiries      99992 non-null  floa

In [152]:
traindata.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
count,99992.000000,9.999200e+04,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000,99992.000000
mean,33.275302,1.764263e+05,4198.524091,17.092167,22.475988,72.470878,3.010041,21.069226,13.263781,10.393515,27.672354,1426.232797,32.285141,221.134191,1403.225371,195.755021,403.121113
std,10.764549,1.429675e+06,3187.152870,117.409489,129.062455,466.440966,62.650385,14.860600,6.202003,6.512789,68.785564,1155.174399,5.116877,99.684181,8306.364896,136.847341,175.477742
min,14.000000,7.005930e+03,303.645417,-1.000000,0.000000,1.000000,-100.000000,-5.000000,-2.000000,-5.010000,0.000000,0.230000,20.000000,1.000000,0.000000,14.528879,92.841401
25%,24.000000,1.945549e+04,1625.597917,3.000000,4.000000,8.000000,1.000000,10.000000,9.000000,5.500000,3.000000,566.050000,28.052567,144.000000,30.306660,97.630668,284.538757
50%,33.000000,3.757861e+04,3097.008333,6.000000,5.000000,13.000000,3.000000,18.000000,14.000000,9.340000,6.000000,1166.080000,32.305784,219.000000,69.266164,152.064614,338.449369
75%,42.000000,7.279092e+04,5961.745000,7.000000,7.000000,20.000000,5.000000,28.000000,18.000000,14.680000,10.250000,1946.810000,36.496462,302.000000,161.224249,257.568430,465.449084
max,56.000000,2.419806e+07,15204.633333,1798.000000,1499.000000,5797.000000,1496.000000,67.000000,28.000000,29.980000,602.250000,4998.070000,50.000000,404.000000,82331.000000,1005.847285,1349.264887


In [153]:
traindata['Num_Bank_Accounts'].describe()

count    99992.000000
mean        17.092167
std        117.409489
min         -1.000000
25%          3.000000
50%          6.000000
75%          7.000000
max       1798.000000
Name: Num_Bank_Accounts, dtype: float64

In [154]:
traindata['Num_Bank_Accounts'].replace(-1,0, inplace=True)

In [155]:
traindata['Num_Bank_Accounts'].value_counts().sort_values(ascending=False).head(20)

6       12993
7       12823
8       12765
4       12186
5       12118
3       11950
9        5443
10       5247
1        4490
0        4349
2        4304
11          9
803         7
1668        5
105         5
791         5
1033        4
43          4
34          4
1481        4
Name: Num_Bank_Accounts, dtype: int64

We can see that the number of bank accounts range from 0 to 1798, from the value counts, we can say that the value greater than 11 is certainly an error

In [156]:
traindata['Num_Bank_Accounts']=traindata['Num_Bank_Accounts'].apply(lambda x: np.nan if x>11 else x)
traindata['Num_Bank_Accounts']=traindata.groupby('Customer_ID')['Num_Bank_Accounts'].apply(lambda x: x.fillna(x.mode()[0]))

C:\Users\user\AppData\Local\Temp\ipykernel_16784\3591632930.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  traindata['Num_Bank_Accounts']=traindata.groupby('Customer_ID')['Num_Bank_Accounts'].apply(lambda x: x.fillna(x.mode()[0]))


In [157]:
traindata['Num_Bank_Accounts']

0        3.0
1        3.0
2        3.0
3        3.0
4        3.0
        ... 
99995    4.0
99996    4.0
99997    4.0
99998    4.0
99999    4.0
Name: Num_Bank_Accounts, Length: 99992, dtype: float64

In [158]:
traindata['Num_Credit_Card'].describe()

count    99992.000000
mean        22.475988
std        129.062455
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max       1499.000000
Name: Num_Credit_Card, dtype: float64

In [159]:
traindata['Num_Credit_Card'].value_counts().sort_values(ascending=False).head(15)

5      18459
7      16615
6      16559
4      14030
3      13269
8       4956
10      4860
9       4643
2       2149
1       2132
11        36
0         13
849        8
852        7
159        6
Name: Num_Credit_Card, dtype: int64

We can see that the number of credit cards range from 0 to 1499, from the value counts, we can say that the value greater than 11 is certainly an error

In [160]:
traindata['Num_Credit_Card']=traindata['Num_Credit_Card'].apply(lambda x: np.nan if x>11 else x)
traindata['Num_Credit_Card']=traindata.groupby('Customer_ID')['Num_Credit_Card'].apply(lambda x: x.fillna(x.mode()[0]))

C:\Users\user\AppData\Local\Temp\ipykernel_16784\3241945591.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  traindata['Num_Credit_Card']=traindata.groupby('Customer_ID')['Num_Credit_Card'].apply(lambda x: x.fillna(x.mode()[0]))


In [161]:
traindata['Num_Credit_Card'].isna().sum()

0

In [162]:
traindata['Num_Credit_Card'].describe()

count    99992.000000
mean         5.533883
std          2.067084
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max         11.000000
Name: Num_Credit_Card, dtype: float64

In [163]:
traindata['Interest_Rate'].describe()

count    99992.000000
mean        72.470878
std        466.440966
min          1.000000
25%          8.000000
50%         13.000000
75%         20.000000
max       5797.000000
Name: Interest_Rate, dtype: float64

Assuming that the realistic interest value is upto 50%, imputing all other values

In [164]:
traindata['Interest_Rate']=traindata['Interest_Rate'].apply(lambda x: np.nan if x>50 else x)
traindata['Interest_Rate']=traindata.groupby('Customer_ID')['Interest_Rate'].apply(lambda x: x.fillna(x.mode()[0]))

C:\Users\user\AppData\Local\Temp\ipykernel_16784\1369711673.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  traindata['Interest_Rate']=traindata.groupby('Customer_ID')['Interest_Rate'].apply(lambda x: x.fillna(x.mode()[0]))


In [165]:
traindata['Interest_Rate'].describe()

count    99992.000000
mean        14.532283
std          8.741651
min          1.000000
25%          7.000000
50%         13.000000
75%         20.000000
max         34.000000
Name: Interest_Rate, dtype: float64

In [166]:
traindata['Interest_Rate'].isna().sum()

0

In [167]:
traindata['Num_of_Loan'].describe()

count    99992.000000
mean         3.010041
std         62.650385
min       -100.000000
25%          1.000000
50%          3.000000
75%          5.000000
max       1496.000000
Name: Num_of_Loan, dtype: float64

In [168]:
traindata['Num_of_Loan'].value_counts().sort_values(ascending=False).head(15)

 3.0       15104
 2.0       15024
 4.0       14743
 0.0       10930
 1.0       10606
 6.0        7803
 7.0        7344
 5.0        7197
-100.0      3876
 9.0        3702
 8.0        3191
 1150.0        4
 288.0         3
 430.0         3
 773.0         3
Name: Num_of_Loan, dtype: int64

In [169]:
traindata['Num_of_Loan']=traindata['Num_of_Loan'].apply(lambda x: np.nan if x<0 or x>10 else x)
traindata['Num_of_Loan']=traindata.groupby('Customer_ID')['Num_of_Loan'].apply(lambda x: x.fillna(x.mode()[0]))

C:\Users\user\AppData\Local\Temp\ipykernel_16784\2498280471.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  traindata['Num_of_Loan']=traindata.groupby('Customer_ID')['Num_of_Loan'].apply(lambda x: x.fillna(x.mode()[0]))


In [170]:
traindata['Num_of_Loan'].isna().sum()

0

In [171]:
traindata['Num_of_Loan'].describe()

count    99992.000000
mean         3.533003
std          2.446416
min          0.000000
25%          2.000000
50%          3.000000
75%          5.000000
max          9.000000
Name: Num_of_Loan, dtype: float64

In [172]:
traindata['Num_of_Delayed_Payment']=traindata['Num_of_Delayed_Payment'].apply(lambda x: np.nan if x<0 else x)
traindata['Num_of_Delayed_Payment']=traindata.groupby('Customer_ID')['Num_of_Delayed_Payment'].apply(lambda x: x.fillna(0))

C:\Users\user\AppData\Local\Temp\ipykernel_16784\1895700395.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  traindata['Num_of_Delayed_Payment']=traindata.groupby('Customer_ID')['Num_of_Delayed_Payment'].apply(lambda x: x.fillna(0))


In [173]:
traindata['Num_of_Delayed_Payment'].describe()

count    99992.000000
mean        13.265541
std          6.197992
min          0.000000
25%          9.000000
50%         14.000000
75%         18.000000
max         28.000000
Name: Num_of_Delayed_Payment, dtype: float64

In [174]:
traindata['Num_of_Delayed_Payment'].isna().sum()

0

In [175]:
traindata['Total_EMI_per_month'].describe()

count    99992.000000
mean      1403.225371
std       8306.364896
min          0.000000
25%         30.306660
50%         69.266164
75%        161.224249
max      82331.000000
Name: Total_EMI_per_month, dtype: float64

In [176]:
totalemi=traindata['Total_EMI_per_month'].value_counts().sort_values(ascending=False)

In [177]:
traindata['Total_EMI_per_month']=traindata.groupby('Customer_ID')['Total_EMI_per_month'].transform(lambda x: x.mode()[0])

In [178]:
traindata['Total_EMI_per_month'].describe()

count    99992.000000
mean       105.546711
std        125.814509
min          0.000000
25%         29.045533
50%         66.052460
75%        145.584994
max       1779.103254
Name: Total_EMI_per_month, dtype: float64

In [179]:
traindata["Delay_from_due_date"].describe()

count    99992.000000
mean        21.069226
std         14.860600
min         -5.000000
25%         10.000000
50%         18.000000
75%         28.000000
max         67.000000
Name: Delay_from_due_date, dtype: float64

In [180]:
traindata.isna().sum()

Customer_ID                 0
Month                       0
Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64

In [181]:
traindata.shape

(99992, 24)

# EDA

In [182]:
traindata.drop(['Customer_ID'], axis=1, inplace=True)

In [183]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99992 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Month                     99992 non-null  category
 1   Age                       99992 non-null  float64 
 2   Occupation                99992 non-null  category
 3   Annual_Income             99992 non-null  float64 
 4   Monthly_Inhand_Salary     99992 non-null  float64 
 5   Num_Bank_Accounts         99992 non-null  float64 
 6   Num_Credit_Card           99992 non-null  float64 
 7   Interest_Rate             99992 non-null  float64 
 8   Num_of_Loan               99992 non-null  float64 
 9   Delay_from_due_date       99992 non-null  int64   
 10  Num_of_Delayed_Payment    99992 non-null  float64 
 11  Changed_Credit_Limit      99992 non-null  float64 
 12  Num_Credit_Inquiries      99992 non-null  float64 
 13  Credit_Mix                99992 non-null  cate

In [184]:
traindata.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,January,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3,4.0,11.27,4.0,Good,809.98,26.822620,265.0,No,49.574949,98.35409,High_spent_Small_value_payments,304.555294,Good
1,February,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,-1,4.0,11.27,4.0,Good,809.98,31.944960,265.0,No,49.574949,98.35409,Low_spent_Large_value_payments,304.555294,Good
2,March,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3,4.0,11.27,4.0,Good,809.98,28.609352,267.0,No,49.574949,98.35409,Low_spent_Medium_value_payments,304.555294,Good
3,April,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,5,4.0,11.27,4.0,Good,809.98,31.377862,268.0,No,49.574949,98.35409,Low_spent_Small_value_payments,304.555294,Good
4,May,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,6,4.0,11.27,4.0,Good,809.98,24.797347,269.0,No,49.574949,98.35409,High_spent_Medium_value_payments,304.555294,Good


In [185]:
plt.pie(traindata['Credit_Score'].value_counts(), labels=traindata['Credit_Score'].value_counts().index, autopct='%.1f')
plt.show()

In [186]:
traindata.shape

(99992, 23)

# Data Preprocessing

In [187]:
cat_df=traindata.select_dtypes('category')

In [188]:
num_df=traindata.select_dtypes(exclude='category')

In [189]:
cols=cat_df.columns

In [190]:
cols

Index(['Month', 'Occupation', 'Credit_Mix', 'Payment_of_Min_Amount',
       'Payment_Behaviour', 'Credit_Score'],
      dtype='object')

In [191]:
for col in cols:
    print(traindata[col].unique())

['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']
Categories (8, object): ['April', 'August', 'February', 'January', 'July', 'June', 'March', 'May']
['Scientist', 'Teacher', 'Engineer', 'Entrepreneur', 'Developer', ..., 'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect']
Length: 15
Categories (15, object): ['Accountant', 'Architect', 'Developer', 'Doctor', ..., 'Musician', 'Scientist', 'Teacher', 'Writer']
['Good', 'Standard', 'Bad']
Categories (3, object): ['Bad', 'Good', 'Standard']
['No', 'NM', 'Yes']
Categories (3, object): ['NM', 'No', 'Yes']
['High_spent_Small_value_payments', 'Low_spent_Large_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Small_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Large_value_payments']
Categories (6, object): ['High_spent_Large_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Small_value_payments', 'Low_spent_Large_value_payments', 'Low_spent_Medium_value_payments', 'L

### Encoding

In [192]:
ohe=OneHotEncoder()

In [193]:
traindata['Credit_Mix']=traindata['Credit_Mix'].map({'Bad':0, 'Standard':1, 'Good':2})

In [196]:
traindata['Credit_Score']=pd.Categorical(traindata['Credit_Score'])
traindata['Credit_Score']=traindata['Credit_Score'].cat.codes
#encodedcreditscore=ohe.fit_transform(traindata['Credit_Score'].values.reshape(-1,1)).toarray()
#encodeddf=pd.DataFrame(encodedcreditscore, columns=ohe.get_feature_names_out(['Credit_Score']))
#traindata=pd.concat([traindata, encodeddf], axis=1)


In [197]:
traindata.drop(['Credit_Score'], axis=1, inplace=True)

In [198]:
traindata['Credit_Mix']=traindata['Credit_Mix'].astype('int')


In [199]:
LE=LabelEncoder()

In [200]:
OccupationCounts=traindata['Occupation'].value_counts()

In [201]:
OccupationCounts

Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Accountant       6744
Scientist        6744
Developer        6720
Media_Manager    6712
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: Occupation, dtype: int64

In [202]:
traindata['Occupation']=traindata['Occupation'].map(OccupationCounts)

In [203]:
traindata['Month']=LE.fit_transform(traindata['Month'])

In [204]:
dummy_df=pd.get_dummies(traindata[['Payment_Behaviour', 'Payment_of_Min_Amount']], drop_first=True)

In [205]:
traindata=pd.concat([traindata, dummy_df], axis=1)
traindata.drop(['Payment_Behaviour', 'Payment_of_Min_Amount'], axis=1, inplace=True)

In [206]:
traindata.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes
0,3,23.0,6744,19114.12,1824.843333,3.0,4.0,3.0,4.0,3,4.0,11.27,4.0,2,809.98,26.822620,265.0,49.574949,98.35409,304.555294,0,1,0,0,0,1,0
1,2,23.0,6744,19114.12,1824.843333,3.0,4.0,3.0,4.0,-1,4.0,11.27,4.0,2,809.98,31.944960,265.0,49.574949,98.35409,304.555294,0,0,1,0,0,1,0
2,6,23.0,6744,19114.12,1824.843333,3.0,4.0,3.0,4.0,3,4.0,11.27,4.0,2,809.98,28.609352,267.0,49.574949,98.35409,304.555294,0,0,0,1,0,1,0
3,0,23.0,6744,19114.12,1824.843333,3.0,4.0,3.0,4.0,5,4.0,11.27,4.0,2,809.98,31.377862,268.0,49.574949,98.35409,304.555294,0,0,0,0,1,1,0
4,7,23.0,6744,19114.12,1824.843333,3.0,4.0,3.0,4.0,6,4.0,11.27,4.0,2,809.98,24.797347,269.0,49.574949,98.35409,304.555294,1,0,0,0,0,1,0


In [207]:
traindata.dtypes

Month                                                   int32
Age                                                   float64
Occupation                                              int64
Annual_Income                                         float64
Monthly_Inhand_Salary                                 float64
Num_Bank_Accounts                                     float64
Num_Credit_Card                                       float64
Interest_Rate                                         float64
Num_of_Loan                                           float64
Delay_from_due_date                                     int64
Num_of_Delayed_Payment                                float64
Changed_Credit_Limit                                  float64
Num_Credit_Inquiries                                  float64
Credit_Mix                                              int32
Outstanding_Debt                                      float64
Credit_Utilization_Ratio                              float64
Credit_H

### Data Splitting

In [208]:
X=traindata

In [209]:
y=encodedcreditscore

In [210]:
sm=SMOTE()
X,y= sm.fit_resample(X,y)

In [211]:
X.shape

(159507, 27)

In [212]:
#y.value_counts()

In [213]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=1)

### Scaling

In [214]:
num_columns=num_df.columns

In [215]:
scalar = PowerTransformer()

In [216]:
X_train[num_columns]=scalar.fit_transform(X_train[num_columns])

In [217]:
import numpy as np

In [218]:
X_test[num_columns]=scalar.transform(X_test[num_columns])

In [219]:
# Model Building

### Neural Network

def VisualizeTrainResult(history, epochs):
    acc=history.history['accuracy']
    val_acc=history.history['val_accuracy']
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    plt.figure(figsize=(16,6))
    plt.subplot(1,2,1)
    plt.plot(acc, label='Training MSE')
    plt.plot(val_acc, label='Validation_MSE')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Accuracy')
    plt.subplot(1,2,2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()










In [220]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from tensorflow.keras import callbacks


In [221]:
##### Model 1

model = Sequential()

model.add(Dense(1024,activation='relu',input_dim=X_train.shape[1]))
model.add(Dropout(0.4))
# model.add(BatchNormalization())

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
# model.add(BatchNormalization())

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(8, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Dense(3, activation="softmax"))

In [222]:
LR=callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, cooldown=1, verbose=1)
early_stopping=callbacks.EarlyStopping(monitor='val_loss', patience=10)
model.compile(optimizer=Adam(learning_rate=0.2), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 1024)              28672     
                                                                 
 dropout_15 (Dropout)        (None, 1024)              0         
                                                                 
 dense_19 (Dense)            (None, 512)               524800    
                                                                 
 dropout_16 (Dropout)        (None, 512)               0         
                                                                 
 dense_20 (Dense)            (None, 256)               131328    
                                                                 
 dropout_17 (Dropout)        (None, 256)               0         
                                                                 
 batch_normalization_9 (Bat  (None, 256)              

In [233]:
epochs=70
history = model.fit(
    np.array(X_train.values),y_train,
    #np.array(X_train.values),y_train,
    epochs=epochs,
    validation_split=0.2,
    batch_size=128,
    callbacks=[early_stopping, LR]
)

Epoch 1/70


ValueError: in user code:

    File "C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_wei

In [228]:
y_pred=model.predict(X_test)

997/997 [==============================] - 6s 5ms/step


In [232]:
print('{:34}|{}'.format('Raw Model Predictions', 'True Labels'))
for i,pred in enumerate(y_pred):
    print('{}|{}'.format(pred, y_test[i]))

Raw Model Predictions             |True Labels
[0.3199055  0.37242803 0.3076664 ]|[0 0 1]
[0.32002294 0.3721992  0.30777788]|[1 0 0]
[0.3199067  0.3724258  0.30766752]|[0 0 1]
[0.31990507 0.3724289  0.307666  ]|[1 0 0]
[0.32000458 0.37223494 0.30776045]|[1 0 0]
[0.31970754 0.3728141  0.3074783 ]|[0 1 0]
[0.31991982 0.37240016 0.30767998]|[0 0 1]
[0.31986168 0.37251356 0.30762482]|[0 1 0]
[0.32015675 0.37193847 0.30790472]|[0 1 0]
[0.31970528 0.3728185  0.3074762 ]|[1 0 0]
[0.31990615 0.37242678 0.307667  ]|[1 0 0]
[0.31992266 0.37239465 0.30768272]|[1 0 0]
[0.31970525 0.37281862 0.3074762 ]|[1 0 0]
[0.31992105 0.3723979  0.3076811 ]|[0 1 0]
[0.31990686 0.37242544 0.30766767]|[0 1 0]
[0.31992668 0.37238678 0.3076865 ]|[1 0 0]
[0.3201277  0.37199506 0.30787727]|[1 0 0]
[0.31989    0.3724584  0.30765164]|[0 1 0]
[0.31988725 0.3724637  0.3076491 ]|[1 0 0]
[0.31970507 0.37281895 0.30747604]|[1 0 0]
[0.31991887 0.37240198 0.3076791 ]|[0 0 1]
[0.31990477 0.37242958 0.30766574]|[1 0 0]
[0.3199

[0.3200083  0.37222776 0.3077639 ]|[0 0 1]
[0.31990835 0.37242264 0.30766907]|[0 1 0]
[0.31990564 0.37242785 0.30766654]|[0 0 1]
[0.320085   0.37207836 0.30783662]|[0 1 0]
[0.31992283 0.37239432 0.30768287]|[1 0 0]
[0.31970447 0.37282002 0.30747548]|[1 0 0]
[0.31990442 0.3724302  0.3076654 ]|[0 1 0]
[0.31983796 0.3725598  0.3076023 ]|[0 1 0]
[0.31995377 0.37233397 0.30771223]|[1 0 0]
[0.31991953 0.37240073 0.3076797 ]|[1 0 0]
[0.3198217  0.3725914  0.30758688]|[1 0 0]
[0.3198597  0.37251732 0.30762297]|[0 0 1]
[0.31994587 0.3723495  0.30770472]|[1 0 0]
[0.32015476 0.37194237 0.30790287]|[1 0 0]
[0.3198394  0.37255695 0.30760363]|[0 1 0]
[0.31985968 0.3725175  0.30762294]|[1 0 0]
[0.31990457 0.37242997 0.30766556]|[1 0 0]
[0.32008255 0.37208313 0.3078344 ]|[1 0 0]
[0.3201282  0.37199408 0.30787772]|[0 1 0]
[0.320023   0.37219915 0.3077779 ]|[1 0 0]
[0.32000595 0.37223238 0.30776167]|[1 0 0]
[0.31992397 0.37239206 0.3076839 ]|[0 0 1]
[0.32008502 0.37207824 0.30783674]|[0 1 0]
[0.32002312

[0.32015762 0.3719369  0.30790555]|[0 1 0]
[0.3199622  0.3723177  0.30772015]|[0 1 0]
[0.31996113 0.3723197  0.30771917]|[1 0 0]
[0.3199229  0.37239417 0.30768293]|[0 1 0]
[0.31983605 0.37256333 0.30760056]|[0 1 0]
[0.31970733 0.37281454 0.30747813]|[0 1 0]
[0.3200061  0.37223205 0.30776185]|[0 0 1]
[0.31994691 0.3723474  0.30770567]|[0 0 1]
[0.3197082  0.3728129  0.30747893]|[0 1 0]
[0.31970525 0.37281862 0.3074762 ]|[0 0 1]
[0.3199054  0.3724282  0.30766633]|[0 0 1]
[0.31983617 0.37256318 0.30760065]|[1 0 0]
[0.31994477 0.37235153 0.30770367]|[1 0 0]
[0.32000834 0.37222782 0.3077639 ]|[0 0 1]
[0.31994832 0.3723447  0.30770698]|[0 1 0]
[0.32002535 0.3721946  0.3077801 ]|[0 1 0]
[0.31996053 0.37232083 0.3077186 ]|[0 0 1]
[0.3198862  0.37246567 0.30764812]|[1 0 0]
[0.31988633 0.3724654  0.30764824]|[1 0 0]
[0.3199622  0.37231758 0.30772018]|[0 1 0]
[0.32000566 0.37223294 0.30776143]|[0 0 1]
[0.31996235 0.37231728 0.3077203 ]|[0 1 0]
[0.31992427 0.37239146 0.30768418]|[0 0 1]
[0.31970495

[0.31992167 0.37239668 0.3076817 ]|[0 1 0]
[0.3198867 0.3724647 0.3076486]|[1 0 0]
[0.32012874 0.3719931  0.3078782 ]|[0 1 0]
[0.32000613 0.372232   0.3077619 ]|[0 0 1]
[0.31990397 0.372431   0.307665  ]|[1 0 0]
[0.3201313  0.37198815 0.30788058]|[0 0 1]
[0.3199466  0.37234804 0.30770537]|[1 0 0]
[0.31983715 0.37256122 0.30760157]|[0 0 1]
[0.31985867 0.3725193  0.30762202]|[1 0 0]
[0.31992102 0.37239793 0.3076811 ]|[0 1 0]
[0.32002607 0.3721931  0.3077808 ]|[0 1 0]
[0.3199051  0.37242886 0.307666  ]|[0 1 0]
[0.32000604 0.37223214 0.3077618 ]|[1 0 0]
[0.31994644 0.37234834 0.30770522]|[1 0 0]
[0.31986207 0.3725128  0.30762514]|[0 1 0]
[0.31992227 0.3723955  0.3076823 ]|[0 1 0]
[0.3200044  0.37223533 0.30776027]|[1 0 0]
[0.3199215  0.37239698 0.30768153]|[0 1 0]
[0.31988996 0.37245852 0.30765158]|[0 1 0]
[0.31970707 0.3728151  0.3074779 ]|[0 0 1]
[0.31970614 0.3728169  0.30747703]|[0 0 1]
[0.31992117 0.37239754 0.30768123]|[0 0 1]
[0.31988904 0.3724602  0.30765074]|[0 1 0]
[0.32008505 0.

[0.31990802 0.37242317 0.30766878]|[0 1 0]
[0.32001787 0.37220913 0.307773  ]|[1 0 0]
[0.31986034 0.37251613 0.30762354]|[1 0 0]
[0.31996208 0.37231782 0.3077201 ]|[0 1 0]
[0.32013005 0.37199053 0.30787942]|[0 1 0]
[0.3198589  0.37251884 0.30762222]|[1 0 0]
[0.31992304 0.3723939  0.30768305]|[0 0 1]
[0.31970844 0.37281242 0.30747914]|[0 1 0]
[0.32008263 0.37208292 0.30783445]|[0 0 1]
[0.32015592 0.37194008 0.30790395]|[0 1 0]
[0.32013053 0.37198952 0.3078799 ]|[0 0 1]
[0.32000792 0.37222847 0.30776358]|[0 1 0]
[0.31988937 0.37245956 0.30765107]|[0 1 0]
[0.32002527 0.37219477 0.30778   ]|[0 0 1]
[0.32002297 0.3721992  0.30777788]|[1 0 0]
[0.32002506 0.37219512 0.30777982]|[0 1 0]
[0.31990486 0.37242937 0.3076658 ]|[1 0 0]
[0.3199252  0.37238967 0.30768508]|[0 1 0]
[0.32000628 0.37223166 0.30776203]|[0 0 1]
[0.32015768 0.37193677 0.30790558]|[0 1 0]
[0.3199216  0.37239674 0.30768165]|[0 1 0]
[0.31990418 0.3724306  0.3076652 ]|[1 0 0]
[0.3199185  0.37240273 0.30767876]|[0 1 0]
[0.31970674

...

[0.31992194 0.3723961  0.30768195]|[0 1 0]
[0.3200232  0.37219867 0.3077781 ]|[0 0 1]
[0.31992397 0.37239215 0.3076839 ]|[0 0 1]
[0.32015446 0.3719429  0.30790257]|[0 0 1]
[0.3201277  0.371995   0.30787724]|[0 0 1]
[0.31970567 0.37281775 0.3074766 ]|[1 0 0]
[0.32003197 0.37218156 0.3077864 ]|[0 1 0]
[0.31988713 0.37246388 0.307649  ]|[0 0 1]
[0.31992328 0.3723934  0.3076833 ]|[0 0 1]
[0.31990543 0.37242824 0.30766633]|[0 0 1]
[0.31983647 0.37256262 0.30760092]|[1 0 0]
[0.31990522 0.37242863 0.30766612]|[0 1 0]
[0.3197054  0.37281832 0.3074763 ]|[0 1 0]
[0.32000488 0.3722344  0.30776072]|[1 0 0]
[0.31988916 0.37245992 0.30765086]|[0 1 0]
[0.32000786 0.37222868 0.3077635 ]|[0 1 0]
[0.3200088  0.3722268  0.30776438]|[0 1 0]
[0.31988657 0.37246495 0.30764848]|[0 0 1]
[0.3199187  0.3724023  0.30767894]|[0 1 0]
[0.31985888 0.3725189  0.30762216]|[0 1 0]
[0.31990498 0.37242907 0.30766594]|[0 0 1]
[0.32012782 0.37199488 0.30787736]|[1 0 0]
[0.32009968 0.37204972 0.30785066]|[1 0 0]
[0.3199045 

[0.31990424 0.37243053 0.30766526]|[1 0 0]
[0.31970513 0.37281874 0.30747607]|[0 1 0]
[0.3199616  0.37231877 0.3077196 ]|[0 1 0]
[0.31991878 0.37240222 0.307679  ]|[0 1 0]
[0.32015622 0.37193966 0.3079042 ]|[0 1 0]
[0.3198877  0.37246278 0.30764952]|[1 0 0]
[0.31992638 0.37238747 0.30768612]|[0 1 0]
[0.31994718 0.3723469  0.30770594]|[0 1 0]
[0.31996033 0.3723212  0.30771843]|[0 0 1]
[0.31970504 0.37281898 0.307476  ]|[1 0 0]
[0.31970707 0.3728151  0.3074779 ]|[0 0 1]
[0.3198391  0.37255752 0.30760336]|[0 1 0]
[0.32015675 0.37193853 0.30790472]|[0 1 0]
[0.320156 0.37194  0.307904]|[0 0 1]
[0.31983718 0.37256122 0.30760157]|[0 0 1]
[0.3200822  0.37208375 0.30783406]|[1 0 0]
[0.31988668 0.37246475 0.30764857]|[1 0 0]
[0.31992173 0.37239647 0.30768177]|[0 0 1]
[0.31992373 0.37239257 0.3076837 ]|[0 0 1]
[0.3198361  0.37256327 0.3076006 ]|[1 0 0]
[0.32002404 0.37219706 0.3077789 ]|[0 1 0]
[0.31994537 0.3723503  0.30770424]|[1 0 0]
[0.31970695 0.37281528 0.30747777]|[0 0 1]
[0.3199608  0.372

[0.32011318 0.37202334 0.30786347]|[1 0 0]
[0.31991273 0.37241402 0.30767328]|[1 0 0]
[0.32012776 0.37199494 0.3078773 ]|[0 1 0]
[0.31990415 0.3724307  0.30766514]|[1 0 0]
[0.31992432 0.3723914  0.30768427]|[0 0 1]
[0.32008222 0.37208357 0.30783412]|[1 0 0]
[0.3199622  0.37231755 0.30772015]|[0 1 0]
[0.32012743 0.37199554 0.307877  ]|[1 0 0]
[0.31990743 0.37242445 0.30766815]|[0 1 0]
[0.3197045 0.37282   0.3074755]|[1 0 0]
[0.31988576 0.3724665  0.3076477 ]|[1 0 0]
[0.31990543 0.37242824 0.30766633]|[0 0 1]
[0.3199468 0.3723476 0.3077056]|[0 1 0]
[0.3197058  0.37281752 0.30747673]|[1 0 0]
[0.3199052  0.37242872 0.30766612]|[0 0 1]
[0.32002452 0.37219617 0.30777934]|[1 0 0]
[0.31988746 0.37246326 0.30764925]|[1 0 0]
[0.31983525 0.37256494 0.30759978]|[0 0 1]
[0.31988755 0.3724631  0.30764937]|[0 1 0]
[0.31992346 0.37239316 0.3076834 ]|[1 0 0]
[0.31983674 0.37256217 0.30760118]|[1 0 0]
[0.31991994 0.37239987 0.3076801 ]|[0 0 1]
[0.3199046  0.3724299  0.30766556]|[1 0 0]
[0.31992263 0.372

[0.3199192  0.37240142 0.30767938]|[1 0 0]
[0.31985956 0.37251762 0.3076228 ]|[1 0 0]
[0.32002348 0.37219816 0.30777833]|[1 0 0]
[0.31988776 0.37246266 0.30764955]|[0 1 0]
[0.3198625  0.37251186 0.3076256 ]|[0 0 1]
[0.31994614 0.37234882 0.307705  ]|[0 0 1]
[0.319706   0.3728171  0.30747688]|[0 1 0]
[0.3199038  0.37243143 0.3076648 ]|[1 0 0]
[0.3199052  0.3724287  0.30766612]|[0 0 1]
[0.31996268 0.37231663 0.3077206 ]|[0 0 1]
[0.31996158 0.37231877 0.3077196 ]|[0 1 0]
[0.31986105 0.37251472 0.30762422]|[0 1 0]
[0.31992224 0.3723955  0.3076823 ]|[1 0 0]
[0.32013097 0.37198874 0.3078803 ]|[0 1 0]
[0.31992352 0.37239295 0.30768353]|[1 0 0]
[0.32008225 0.37208363 0.3078341 ]|[1 0 0]
[0.31983623 0.3725631  0.30760068]|[1 0 0]
[0.31988645 0.37246525 0.30764833]|[0 1 0]
[0.3199079  0.37242344 0.30766863]|[0 1 0]
[0.3198869  0.37246436 0.30764875]|[1 0 0]
[0.31991905 0.3724017  0.30767924]|[1 0 0]
[0.32002413 0.37219682 0.30777898]|[1 0 0]
[0.31986102 0.37251484 0.3076242 ]|[0 1 0]
[0.31990469

[0.32012904 0.37199244 0.3078785 ]|[0 0 1]
[0.32012755 0.37199533 0.3078771 ]|[1 0 0]
[0.32000655 0.3722312  0.3077623 ]|[0 0 1]
[0.31991833 0.37240312 0.30767858]|[1 0 0]
[0.31990469 0.3724297  0.30766565]|[0 1 0]
[0.3199062  0.37242672 0.30766705]|[0 0 1]
[0.31992182 0.37239632 0.30768183]|[0 1 0]
[0.32008168 0.37208474 0.3078336 ]|[1 0 0]
[0.31970865 0.37281197 0.30747935]|[0 1 0]
[0.32002443 0.37219632 0.30777925]|[0 0 1]
[0.3198617  0.37251356 0.30762482]|[0 1 0]
[0.32015684 0.37193838 0.30790478]|[0 1 0]
[0.32008177 0.3720845  0.30783364]|[1 0 0]
[0.3200849  0.37207854 0.3078366 ]|[0 0 1]
[0.31992722 0.3723858  0.30768704]|[1 0 0]
[0.3199185  0.37240273 0.30767876]|[1 0 0]
[0.32000595 0.37223235 0.30776173]|[0 1 0]
[0.31996143 0.37231907 0.30771944]|[0 1 0]
[0.31996113 0.3723198  0.30771914]|[0 0 1]
[0.31992376 0.3723925  0.30768374]|[1 0 0]
[0.31970474 0.3728196  0.30747572]|[1 0 0]
[0.319924   0.37239206 0.30768394]|[1 0 0]
[0.31983876 0.3725582  0.30760306]|[0 1 0]
[0.31990618

In [225]:
ytraincategorical=to_categorical(y_train)

In [226]:
ytraincategorical

array([[[1., 0.],
        [1., 0.],
        [0., 1.]],

       [[0., 1.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.],
        [1., 0.]],

       ...,

       [[1., 0.],
        [1., 0.],
        [0., 1.]],

       [[1., 0.],
        [1., 0.],
        [0., 1.]],

       [[0., 1.],
        [1., 0.],
        [1., 0.]]], dtype=float32)